# Creating A Spotify Database from Scratch

 Let's begin by importing needed packages and reading in the Spotify data.

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from IPython.display import HTML
from sqlalchemy import create_engine

In [ ]:
# Read in spotify data into a dataframe
df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')
df.columns
pd.set_option('display.max_columns', 30)
subset = df.sample(5)

In [ ]:
HTML(subset.to_html())

In [ ]:
df[['track_id', 'track_album_id', 'track_album_name']]

The first step in getting this data into a database is normalizing it. Normalization makes the database more efficient and flexible by reducing the amount of redundant data. The database is already in first normal form, as there are no columns that contain multiple data elements; the next step is to determine how to get it into second normal form. Second normal form is when all of the non-key columns are dependent only on the table's primary key. This sounds a bit jargony, I know, but an example may help clarify things. 

A primary key is a value that is unique for each record in the database. In the current scenario, we have a unique track id for every track in our dataset, so we could potentially use this as our primary key. If the database was in second normal form all the other columns in the table would only be dependent on the track id. This is not the case currently, as the present columns are dependent on other values in the database. For example, the track album name is dependent on the track album id. We need to break the table up into smaller tables to fix this problem.

| Table Name  | Contents    |
| ----------- | ----------- |
| tracks       | track id and song characteristics|
| album_name  | album name and album id       |
| album release | album id and release date|
| playlist  | playlist id and playlist characteristics|
| track_playlist | track id and playlist id |
| track_artist | track id and artist name|

In [ ]:
# Create subset dataframes to be turned into individual tables
tracks = df[['track_id', 'track_popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms']]

track_name = df [['track_id', 'track_name']]

track_album = df[['track_album_id', 'track_id']]

album_name = df[['track_album_id', 'track_album_name']]

album_release = df[['track_album_release_date', 'track_album_id']]

playlist = df[['playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre']]

track_playlist = df[['playlist_id', 'track_id']]

track_artist = df[['track_id', 'track_artist']]

playlist_genre = df[['playlist_genre', 'playlist_subgenre']]

In [ ]:
# Create a database to connect to in memory
cnx = sqlite3.connect('spotify.db')

# Create the tables in the database
tracks.to_sql(name='tracks', con=cnx, index = False)
track_name.to_sql(name='track_name', con=cnx, index = False)
playlist.to_sql(name='playlist', con = cnx, index = False)
album_release.to_sql(name='album_release', con = cnx, index = False)
album_name.to_sql(name='album_name', con = cnx, index = False)
track_album.to_sql(name='track_album', con = cnx, index = False)
track_playlist.to_sql(name = 'track_playlist', con = cnx, index = False)
track_artist.to_sql(name= 'track_artist', con = cnx, index = False)

 Load SQL and connect to our new Spotify database.

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///spotify.db

Examine the structure of the SQL database tables.

In [ ]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

In [ ]:
%%sql
SELECT *
FROM tracks
LIMIT 5

In [ ]:
%%sql
SELECT * 
FROM track_name
LIMIT 5

In [ ]:
%%sql
SELECT *
FROM playlist
LIMIT 5

In [ ]:
%%sql
SELECT *
FROM album_release
LIMIT 5

In [ ]:
%%sql
SELECT * 
FROM album_name
LIMIT 5

In [ ]:
%%sql
SELECT * 
FROM track_album
LIMIT 5

Let's try a sample query! I'm interested in the average danceability of songs by playlist genre. What playlist is the most danceable?

In [ ]:
%%sql
SELECT AVG(tracks.danceability) as Danceability, playlist.playlist_genre as Genre
FROM tracks
JOIN playlist ON playlist.playlist_id = track_playlist.playlist_id
JOIN track_playlist ON track_playlist.track_id = tracks.track_id
GROUP BY playlist.playlist_genre
ORDER by AVG(tracks.danceability) DESC

In [ ]:
%%sql
SELECT DISTINCT(track_name.track_name) as TrackName, (tracks.danceability) as Danceability, playlist.playlist_genre as Genre
FROM tracks
JOIN playlist ON playlist.playlist_id = track_playlist.playlist_id
JOIN track_name ON track_name.track_id = tracks.track_id
JOIN track_playlist ON track_playlist.track_id = tracks.track_id
WHERE playlist.playlist_genre = 'rap'
ORDER by (tracks.danceability) DESC
LIMIT 5

It seems a little strange that rap is the most danceable music... I wonder if something else is driving this?

For fun, let's look at the least danceable music! It turns out it's a track called, "Hi, How're You Doin'?" by DREAMS COME TRUE. It's just a 10-second recording of someone saying "Hi, How're You Doin?' The rating seems about right. 

In [ ]:
%%sql
SELECT DISTINCT(track_name.track_name), track_artist.track_artist, tracks.energy
FROM tracks
JOIN track_name ON track_name.track_id = tracks.track_id
JOIN track_artist ON track_artist.track_id = tracks.track_id
ORDER BY tracks.energy DESC
LIMIT 10

The most popular tracks of our time.

In [ ]:
%%sql
SELECT DISTINCT(track_name.track_name), tracks.track_popularity, track_artist.track_artist
FROM tracks
JOIN track_name ON track_name.track_id = tracks.track_id
JOIN track_artist ON track_artist.track_id = tracks.track_id
ORDER BY tracks.track_popularity DESC
LIMIT 10

In [ ]:
%%sql
SELECT AVG(tracks.valence), playlist.playlist_genre
FROM tracks
JOIN track_name ON track_name.track_id = tracks.track_id
JOIN track_artist ON track_artist.track_id = tracks.track_id
JOIN playlist ON playlist.playlist_id = track_playlist.playlist_id
JOIN track_playlist ON track_playlist.track_id = tracks.track_id
GROUP BY playlist.playlist_genre
ORDER BY AVG(tracks.valence) DESC

Use an SQL query to find the names of all playlists that contain instrumentals.

In [ ]:
%%sql 
SELECT playlist.playlist_name
FROM tracks
JOIN track_name ON track_name.track_id = tracks.track_id
JOIN track_artist ON track_artist.track_id = tracks.track_id
JOIN playlist ON playlist.playlist_id = track_playlist.playlist_id
JOIN track_playlist ON track_playlist.track_id = tracks.track_id
GROUP BY playlist.playlist_id
LIMIT 10

In [ ]:
%%sql 
SELECT COUNT(*)
FROM tracks
WHERE tracks.instrumentalness >.50

In [ ]:
%%sql 
SELECT PlaylistName, NumberInstrumentals
FROM (SELECT COUNT(DISTINCT(tracks.track_id)) as NumberInstrumentals, playlist.playlist_name as PlaylistName
      FROM tracks
      JOIN track_playlist ON track_playlist.track_id = tracks.track_id
      JOIN track_name ON track_name.track_id = tracks.track_id
      JOIN track_artist ON track_artist.track_id = tracks.track_id
      JOIN playlist ON playlist.playlist_id = track_playlist.playlist_id
      WHERE tracks.instrumentalness >.50
      GROUP BY playlist.playlist_name
     )
WHERE NumberInstrumentals > 0
ORDER BY NumberInstrumentals DESC
LIMIT 10

Find the total number of playlists containing instrumentals. 

In [ ]:
%%sql 
SELECT COUNT(DISTINCT(PlaylistName))
FROM (SELECT COUNT(DISTINCT(tracks.track_id)) as NumberInstrumentals, playlist.playlist_name as PlaylistName
      FROM tracks
      JOIN track_playlist ON track_playlist.track_id = tracks.track_id
      JOIN track_name ON track_name.track_id = tracks.track_id
      JOIN track_artist ON track_artist.track_id = tracks.track_id
      JOIN playlist ON playlist.playlist_id = track_playlist.playlist_id
      WHERE tracks.instrumentalness >.50
      GROUP BY playlist.playlist_name
     )
WHERE NumberInstrumentals > 0
LIMIT 10